In [1]:
using JSON, HTTP
function get_data(url)
    resp = HTTP.get(url);
    es = String(resp.body)
    return placements = JSON.parse(es)
end

get_data (generic function with 1 method)

In [2]:
url = "https://support.econjobmarket.org/api/placement_data"
placements = get_data(url);

In [3]:
placements[1000]

Dict{String, Any} with 17 entries:
  "to_shortname"          => "Public Policy, Univ of Michigan"
  "to_name"               => "University of Michigan"
  "to_department"         => "Gerald R Ford School of Public Policy"
  "name"                  => "Health; Education; Welfare"
  "category_id"           => 20
  "from_shortname"        => "Economics, Univ of Maryland"
  "to_oid"                => 1351
  "recruiter_type"        => 4
  "description"           => "Academic organization (other than econ, business,…
  "from_oid"              => "20"
  "position_name"         => "Post-Doc"
  "year"                  => 2010
  "aid"                   => 4881
  "to_institution_id"     => 4173
  "postype"               => 6
  "from_institution_id"   => "13"
  "from_institution_name" => "University of Maryland"

In [17]:
function process_data(YEAR_INTERVAL, placements)
    i = 0
    oid_mapping = Dict{}()
    institution_mapping = Dict{}()
    academic = Set{}()
    academic_to = Set{}()
    academic_builder = Set{}()
    sink_builder = Set{}()
        #if in(parse(Int64, placement["year"]), YEAR_INTERVAL)
    for placement in placements
            if in(placement["year"], YEAR_INTERVAL)
            push!(academic, placement["from_institution_name"])
            if placement["position_name"] == "Assistant Professor"
                    push!(academic_to, placement["to_name"])
            end
            oid_mapping[placement["from_oid"]] = placement["from_institution_id"]
            oid_mapping[placement["to_oid"]] = placement["to_institution_id"]
            institution_mapping[placement["from_institution_id"]] = placement["from_institution_name"]
            institution_mapping[placement["to_institution_id"]] = placement["to_name"]
            if placement["postype"] == 1
                push!(academic_builder, placement)
            else
                push!(sink_builder, placement)
            end
        end
    end
 println("Debug: ", length(placements), " returned from api")
    println("Debug: ", length(academic_builder), " academic placements")
    println("Debug: ", length(sink_builder), " sinks")
    println("Debug: ", length(academic_builder)+length(sink_builder), " in all")
        
    tch_sink = Set{}() # sink of teaching universities that do not graduate PhDs
    for key in academic_to
        if !(key in academic)
            push!(tch_sink, key)
        end
    end


    acd_sink = Set{}()
    gov_sink = Set{}()
    pri_sink = Set{}()
    postdoc = Set{}()
    lecturer = Set{}()
    for outcome in sink_builder
        # CODE global academic, other_placements, pri_sink, gov_sink, acd_sink
        if outcome["recruiter_type"] in [6, 7]
            # private sector: for and not for profit
            push!(pri_sink, string(outcome["to_name"], " (private sector)"))
        elseif outcome["recruiter_type"] == 5
            # government institution
            push!(gov_sink, string(outcome["to_name"], " (public sector)"))
        elseif outcome["postype"] == 6
            #postdoc
            push!(postdoc, string(outcome["to_name"], " (postdoc)"))
        elseif outcome["postype"] in [5, 7]
            #postdoc
            push!(lecturer, string(outcome["to_name"], " (lecturer)"))
        else
            # everything else including terminal academic positions
            push!(acd_sink, string(outcome["to_name"], " (academic sink)"))
        end
    end
    return academic_builder, sink_builder, academic, acd_sink, gov_sink, pri_sink, tch_sink, postdoc, lecturer, institution_mapping
end


process_data (generic function with 1 method)

In [15]:
function name_to_oid(name, institution_mapping)
    ## returns an institution_id
    for (key, value) in institution_mapping
        if value == name
            return key
        end
    end
    return "not found"
end

name_to_oid (generic function with 1 method)

In [19]:
YEAR_INTERVAL = 2010:2020
academic_builder, sink_builder, academic, acd_sink, gov_sink, pri_sink, tch_sink, postdoc, lecturer, institution_mapping = process_data(YEAR_INTERVAL, placements);

Debug: 14876 returned from api
Debug: 6036 academic placements
Debug: 6163 sinks
Debug: 12199 in all


In [20]:
println(length(academic_builder), " total assistant professor outcomes")
    println(length(sink_builder), " other outcomes")
    sample_size = length(academic_builder)+length(sink_builder)
    println(sample_size, " total outcomes")
println(length(acd_sink), " Other academic institutions")
println(length(gov_sink), " Government institutions")
println(length(pri_sink), " Private Sector institutions")
println(length(tch_sink), " Teaching institutions")
println(length(postdoc), " Institutions hiring postdocs")
println(length(lecturer), " Institutions hiring sessionals")

6036 total assistant professor outcomes
6163 other outcomes
12199 total outcomes
479 Other academic institutions
142 Government institutions
179 Private Sector institutions
583 Teaching institutions
459 Institutions hiring postdocs
331 Institutions hiring sessionals


In [21]:
name_to_oid("University of British Columbia", institution_mapping)

"57"

In [22]:
academic

Set{Any} with 688 elements:
  "Indian Statistical Institute, Delhi"
  "Eastern Kentucky University"
  "Indian Institute of Technology, Indore"
  "Peking University"
  "University of Jammu"
  "Indiana Institute of Technology"
  "Rutgers University, Newark"
  "University of Cologne"
  "Technische Universität Dresden"
  "Université de Toulon"
  "Leiden University"
  "University of Dundee"
  "University of Leicester"
  "London Business School"
  "University of Bergen"
  "Università degli Studi di Napoli Federico II"
  "University of Waikato"
  "University of Zurich"
  "University of Rhode Island"
  "Universidad Rey Juan Carlos"
  "Ca' Foscari University Venice"
  "Kurukshetra University"
  "EDHEC Business School"
  "Institute of Technology Sligo"
  "University of St Andrews"
  ⋮ 